In [1]:
import numpy as np
import splat
import wisps
import pandas as pd
from scipy import stats
import numba

In [2]:
stype='std'

In [3]:
spex_df=pd.read_pickle(wisps.OUTPUT_FILES+'/'+str(stype)+'spex_sample.pkl' )

In [4]:
spex_df.shape

(2649, 3)

In [5]:
forbidden=['J0148+1202',  'J0331+4130', 'J0338-4409', 'J0343+3155',
' J0344+3204', '0344+3200', 'J0344+3156', 'J0344+3203',
'J0345+3205','J0419+2712', 'J0435+2254','J0438+2519',
'J0448-4432', 'J0448-4432', 'J0501-3341', 'J0512-2949',
'J0610-2151','J0621+6558','J0624-1045', 'J0628-0959',
'J0815+1041','J0822+1700', 'J0935+0019','J0950+0540',
'J1004+5023','J1050-2940','J1132-3018','J1132-3018',
'J1132-3018','J1132-3018','J1132-3018','J1132-3018',
'J1132-3018','J1132-3018', 'J1132-3018','J1132-3018',
'J1138-1314','J1209-3041','J1211-2821','J1224-2744',
'J1257-0204','J1303+2351','J1312+0051','J1317-1427',
'J1325-2128', 'J1420-1752', 'J1423+0116','J1629+1415',
'J1642-2355','J1642-2355','J1659+3515','J1726-1158',
'J1729+4352','J1829+5032','J1839-3744','J1924+5506',
'J1932+0133', 'J1932-3921', 'J1945-4149', 'J2001-3805',
'J2024-3422', 'J2028+6925', 'J2034+6727','J2151-3349']

In [6]:
spex_df

,splat,wisps,shortname
0,SPEX-PRISM spectrum of 2MASS J0000286-124515,anon spectrum,J0000-1245
1,SPEX-PRISM spectrum of 2MASS J0000286-124515,anon spectrum,J0000-1245
2,SPEX-PRISM spectrum of SDSS J000112.18+153535.5,anon spectrum,J0001+1535
3,SPEX-PRISM spectrum of 2MASS J00013044+1010146,anon spectrum,J0001+1010
4,SPEX-PRISM spectrum of WISE J000131.93-084126.9,anon spectrum,J0001-0841
...,...,...,...
2644,SPEX-PRISM spectrum of SSSPM J2356-3426,anon spectrum,J2356-3426
2645,SPEX-PRISM spectrum of 2MASSI J2356547-155310,anon spectrum,J2356-1553
2646,SPEX-PRISM spectrum of WISE J235716.49+122741.8,anon spectrum,J2357+1227
2647,SPEX-PRISM spectrum of 2MASS J23590855+2338326,anon spectrum,J2359+2338


In [7]:
spex_df['shortname']=spex_df.splat.apply(lambda x:x.shortname)

In [8]:
spex_df=(spex_df[~ spex_df.shortname.isin(forbidden)]).reset_index(drop=True)

In [9]:
spex_df.shape

(2566, 3)

In [10]:
def add_new_columns(row):
    return pd.concat([pd.Series(row.wisps.indices), pd.Series(row.wisps.snr)])

In [11]:
indices=pd.DataFrame(spex_df.apply(add_new_columns, axis=1))
#snr=pd.DataFrame(spex_df.apply(add_new_columns, axis=1))

In [12]:
all_spex=splat.searchLibrary(vlm=True, giant=False)

In [13]:
spex_df['data_file']=[x.data_file for x in spex_df.splat]

In [14]:
all_spex.columns=[x.lower() for x in all_spex.columns]

In [15]:
merged=spex_df.merge(all_spex, on='data_file', how='inner')

In [16]:
for c in ['shortname', 'wisps', 'data_file', 'metallicity_class', 'gravity_class_nir', 'sbinary', 'h_2mass', 'j_2mass' ]:
          indices[c]=merged[c]

In [17]:
indices['f_test']=(indices['wisps']).apply(lambda x: x.f_test)
indices['line_chi']=(indices['wisps']).apply(lambda x: x.line_chi)
indices['spex_chi']=(indices['wisps']).apply(lambda x: x.spex_chi)
indices['spt']=(indices['wisps']).apply(lambda x: x.spectral_type)

In [18]:
indices.drop( columns='wisps').to_pickle(wisps.LIBRARIES+'/spex_data_set_table.pkl')